# Style-Color Allocation and Distribution

#### Generated Categories:
1. style categories
2. color categories
3. shop ids
4. bundle ids
5. bundle to style-color

#### Static Values
1. randomly generated intial inventory
2. budget
3. bundle cost
4. desired allocation

### TODO
1. create available bundles
2. represent the cost, in dollars, for not having the required inventory
3. the desired allocation by style could be represented as a seperate cost function, or dollar cost

The allocation of bundles are allocated to maximize the use of the desired budget. Bundles with specific sets of styles and colors are sent to shops.

The amount of style-color inventory to added, to the current inventory, for each shop is represented as a variable. The difference between the desired inventory and the updated inventory is then represented as a cost. This is the second optimization objective.


In [23]:

# This command imports the Gurobi functions and classes.
import os

os.environ['GRB_LICENSE_FILE'] = 'C:\\Users\\rohan.kotwani\\gurobi.lic'

import gurobipy as gp
from gurobipy import GRB


import pandas as pd
# from pylab import *
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(1)

styles = gp.tuplelist(["A","B","C"])
colors=gp.tuplelist(["R", "B", "G"])
shops = gp.tuplelist(["1","2","3"])
bundles = gp.tuplelist(["1","2","3","4"])


bundles_static={
    "1":{"AR":1,"AB":1,"AG":1,"BR":1,"BB":1,"BG":1,"CR":1,"CB":1,"CG":1},
    "2":{"AR":3,"AB":3,"AG":3,"BR":0,"BB":0,"BG":0,"CR":0,"CB":0,"CG":0},
    "3":{"AR":0,"AB":0,"AG":0,"BR":3,"BB":3,"BG":3,"CR":0,"CB":0,"CG":0},
    "4":{"AR":0,"AB":0,"AG":0,"BR":0,"BB":0,"BG":0,"CR":3,"CB":3,"CG":3}
}
bundles_gp = gp.multidict(bundles_static)

budget = 1000
bundle_cost = 10
desired_allocation = 40

# availability = {"1":30,"2":40,"3":20,"4":60}


current_inventory = {}
for i,shop in enumerate(shops):
    current_inventory[shop] = {}
    for style in styles:
        for color in colors:
            current_inventory[shop][style+color] = np.random.randint(0,10)
            


bundle_allocation = []
#bundle k1
for k1 in bundles_static.keys():
    #shop+color k2
    for k2 in bundles_static[k1].keys():
        bundle_allocation.append((k1,k2))
        
bundle_allocation_gp = gp.tuplelist(bundle_allocation)

inventory_allocation = []
for shop in shops:
    for bundle in bundles:
        inventory_allocation.append((shop,bundle))
        
inventory_allocation_gp = gp.tuplelist(inventory_allocation)


model = gp.Model("shirt_alloc")

b_var = model.addVars(bundles, vtype=GRB.INTEGER, name="b_var")
ba_var = model.addVars(bundle_allocation_gp, vtype=GRB.INTEGER, name="ba_var")

#constraint on budget & bundles
v = model.addVar(lb=0,ub=budget, name="v")
budget_req = model.addConstr(v == budget-b_var.sum('*','*')*bundle_cost,name='budget_req')

#can only add a maximum of 20 style colors
ia_var = model.addVars(inventory_allocation_gp, lb=0,ub=20, vtype=GRB.INTEGER, name="ia_var")

for bundle in bundles_static:
    for style in styles:
        for color in colors:
            #constraint on bundle and allocation
            model.addConstr(b_var.sum(bundle)*bundles_static[bundle][style+color]==ba_var.sum(bundle,style+color))
    #make sure that the all distributor bundles are sent to shops
    model.addConstr(b_var.sum(bundle)==ia_var.sum("*",bundle))

#bundle cost objective
cost_obj = gp.quicksum(ba_var)
     
#efficiency of allocation
allocation_eff = []
for shop in shops:
    for style in styles:
        for color in colors:
            sc_units = current_inventory[shop][style+color]
            for bundle in bundles:
                sc_units += ia_var.sum(shop,bundle)*bundles_static[bundle][style+color] 
            
            #allocated bundles*number of shirts in bundle_id for each style color
            loss = sc_units - desired_allocation
            #ensure it does not go over the desired allocation
            model.addConstr(loss <= 0)
            allocation_eff.append(loss)

allocation_obj = gp.quicksum(allocation_eff)
            
model.ModelSense = GRB.MAXIMIZE

# m.setObjective(TSS)
model.setObjectiveN(allocation_obj, index=0, priority=1, name='ALLOC_EFF')
model.setObjectiveN(cost_obj, index=1, priority=2, name='MAX_COST')

In [24]:
model.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 68 rows, 53 columns and 129 nonzeros
Model fingerprint: 0x7eebe614
Variable types: 1 continuous, 52 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 9e+00]
  Bounds range     [2e+01, 1e+03]
  RHS range        [3e+01, 1e+03]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 58 rows and 40 columns
Presolve time: 0.00s
Presolved: 10 rows and 13 columns
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (MAX_COST) ...


In [27]:
current_inventory_df = pd.DataFrame(columns=('shop_id','style','color','sc_units'))
updated_inventory_df = pd.DataFrame(columns=('shop_id','style','color','sc_units'))

i=0
for shop in shops:
    for style in styles:
        for color in colors:
            sc_units = current_inventory[shop][style+color]
            current_inventory_df.loc[i]=[shop,style,color,sc_units]
            
            for bundle in bundles:
                sc_units += ia_var[shop,bundle].x*bundles_static[bundle][style+color] 
                    
            updated_inventory_df.loc[i]=[shop,style,color,sc_units]
            i+=1
#                 print(ia_var[shop,bundle].x*bundles_static[bundle][style+color] 
#                                +current_inventory[shop][style+color])

# Updated Inventory

In [28]:
pd.set_option('display.max_rows',200)
updated_inventory_df

,shop_id,style,color,sc_units
0,1,A,R,36.0
1,1,A,B,39.0
2,1,A,G,40.0
3,1,B,R,39.0
4,1,B,B,34.0
5,1,B,G,34.0
6,1,C,R,32.0
7,1,C,B,38.0
8,1,C,G,37.0
9,2,A,R,40.0


# Initial Inventory

In [39]:
current_inventory_df

,shop_id,style,color,sc_units
0,1,A,R,5
1,1,A,B,8
2,1,A,G,9
3,1,B,R,5
4,1,B,B,0
5,1,B,G,0
6,1,C,R,1
7,1,C,B,7
8,1,C,G,6
9,2,A,R,9


In [41]:
# import matplotlib.pyplot as plt
# plt.plot(current_inventory_df.sc_units.values,'+')
# plt.plot(updated_inventory_df.sc_units.values,'+')

# plt.plot(updated_inventory_df.sc_units.values-current_inventory_df.sc_units.values,'+')

# Bundles Sent to Shops

In [42]:
import pandas as pd
shop_bundle_df = pd.DataFrame(columns=('shop_id','bundle_id','bundle_units'))

i=0
for shop in shops:
    for bundle in bundles:
        shop_bundle_df.loc[i] = [shop,bundle,ia_var[shop, bundle].x]
        i+=1
shop_bundle_df

,shop_id,bundle_id,bundle_units
0,1,1,19.0
1,1,2,4.0
2,1,3,5.0
3,1,4,4.0
4,2,1,19.0
5,2,2,4.0
6,2,3,5.0
7,2,4,4.0
8,3,1,19.0
9,3,2,4.0


# Total Bundles Allocated

In [43]:
alloc_bundle_df = pd.DataFrame(columns=('bundle_id','alloc_bundles'))
i=0
for bundle in bundles:

    alloc_bundle_df.loc[i] = [bundle,b_var[bundle].x]
    i+=1
alloc_bundle_df

,bundle_id,alloc_bundles
0,1,57.0
1,2,12.0
2,3,14.0
3,4,12.0
